# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [4]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

help(transforms.Normalize)

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [5]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

transforms.Normalize(mean=[.5], std=[.5])
进行归一化mean均值 std标准差
input[channel] = (input[channel] - mean[channel]) / std[channel]
这里是定义了一个函数
transform = transforms.Compose([transforms.ToTensor(), normalize])
这里用到了上面的函数
transforms.ToTensor()
将读入的图片格式（hign，width，channels）转换为（c,h,w）并且归一化0~255位0~1
在进行normalize

 torchvision.datasets.MNIST（root，train = True，transform = None，target_transform = None，download = False ）
 root（string） - 数据集的根目录在哪里MNIST/processed/training.pt 和 MNIST/processed/test.pt存在。
 train（bool，optional） - 如果为True，则创建数据集training.pt，否则创建数据集test.pt。
 download（bool，optional） - 如果为true，则从Internet下载数据集并将其放在根目录中。如果已下载数据集，则不会再次下载。

dataloader = data.DataLoader(dataset, batch_size, shuffle=True, drop_last)
定义如何读取数据
dataset数据库
batch_size一次读取多少行数据
shuffle洗牌即打乱顺序，默认为False
drop_last由于设置了batch_size，所以可能出现无法整除因而余下的数据，设置为True丢弃剩下的数据，默认为False



Then, we define the model, object function and optimizer that we use to classify.

In [6]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Sequential(     #input_size=(1*28*28)
            nn.Conv2d(1, 6, 5, 1, 2), #padding=2保证输入输出尺寸相同
            nn.ReLU(),      #input_size=(6*28*28)
            nn.MaxPool2d(kernel_size=2, stride=2),#output_size=(6*14*14)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),      #input_size=(16*10*10)
            nn.MaxPool2d(2, 2)  #output_size=(16*5*5)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(120, 84),
            nn.ReLU()
        )
        self.fc3 = nn.Linear(84, 10)

    # 定义前向传播过程，输入为x
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # nn.Linear()的输入输出都是维度为一的值，所以要把多维度的tensor展平成一维
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x


    
model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.001,momentum=0.9)

Next, we can start to train and evaluate!

In [7]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    print(epoch,'\n')
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    

  0%|                                                                                          | 0/468 [00:00<?, ?it/s]

0 



  0%|▏                                                                                 | 1/468 [00:00<00:54,  8.57it/s]

1 



  0%|                                                                                          | 0/468 [00:00<?, ?it/s]

...

  0%|▎                                                                                 | 2/468 [00:00<00:38, 12.15it/s]

8 



  0%|▏                                                                                 | 1/468 [00:00<01:21,  5.73it/s]

9 



100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:42<00:00, 11.08it/s]


In [8]:
correct_train = 0
correct_test = 0
total_train = 0
total_test = 0
for images,labels in tqdm(train_loader):
    outputs = model(images)
    _,predicted = torch.max(outputs.data,1)
    total_train += labels.size(0)
    correct_train += (predicted == labels).sum()
    
for images,labels in tqdm(test_loader):
    outputs = model(images)
    _,predicted = torch.max(outputs.data, 1)
    total_test += labels.size(0)
    correct_test += (predicted == labels).sum()

train_accuracy = float(correct_train)/total_train
print('Training accuracy is：%0.4f%%' % ((100 * train_accuracy)))
test_accuracy = float(correct_test)/total_test
print('Testing accuracy is：%0.4f%%' % ((100 * test_accuracy)))

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:04<00:00, 18.31it/s]

Training accuracy is：97.2523%
Testing accuracy is：97.2857%


#### Q5:
Please print the training and testing accuracy.